# Pairwise Comparisons

In [1]:
from pref_voting.pairwise_profiles import PairwiseBallot, PairwiseProfile
from pref_voting.voting_methods import *

In [3]:
from pref_voting.profiles_with_ties import ProfileWithTies
from pref_voting.iterative_methods import instant_runoff_for_truncated_linear_orders

prof = ProfileWithTies([{0:1, 1:1},{0:1, 1:2, 2:3, 3:4}, {0:1, 1:3, 2:3}, {3:2}, {0:1}, {0:1}])
prof.display()

tprof, report = prof.truncate_overvotes()
for r, new_r, count in report: 
    print(f"{r} --> {new_r}: {count}")
tprof.display()
instant_runoff_for_truncated_linear_orders.display(tprof)
    


+-----+---+-----+---+---+---+
|  1  | 1 |  1  | 1 | 1 | 1 |
+-----+---+-----+---+---+---+
| 0 1 | 0 |  0  | 3 | 0 | 0 |
|     | 1 | 1 2 |   |   |   |
|     | 2 |     |   |   |   |
|     | 3 |     |   |   |   |
+-----+---+-----+---+---+---+
( 0  1 )  --> : 1
0 ( 1  2 )  --> 0 : 1


ValueError: min() arg is an empty sequence

In [2]:
A="A"
B="B"
C="C"
D="D"
E="E"

cands = [A, B, C, D, E]

lin_order_ballot = {"A": 1, "B": 2, "C": 3, "D": 4, "E": 5}
complete_ranking_with_ties_ballot = {"A": 1, "B": 1, "C": 3, "D": 4, "E": 4}
bullet_ballot1 = {"A":1}
bullet_ballot2 = {"A":2}
partial_ballot = {"A":1, "C":2}
skipped_rank_ballot = {"A":1, "B":4}
tied_partial_ballot = {"A":1, "B":1, "C":2}
tied_partial_ballot1 = {"A":1, "B":1}
tied_partial_ballot2 = {"A":2, "B":2}
all_tied_ballot1 = {"A":1, "B":1, "C":1, "D":1, "E":1}
all_tied_ballot2 = {"A":2, "B":2, "C":2, "D":2, "E":2}

ballots = [lin_order_ballot, complete_ranking_with_ties_ballot, bullet_ballot1, bullet_ballot2, partial_ballot, skipped_rank_ballot, tied_partial_ballot1, tied_partial_ballot2, all_tied_ballot1, all_tied_ballot2]

In [3]:
def has_skipped_rank(ballot):
    ranks = sorted(set(ballot.values()))
    return ranks != list(range(1, len(ranks)+1))

for b in ballots:
    print(b)
    print("  has skipped rank?", has_skipped_rank(b))
    print()

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
  has skipped rank? False

{'A': 1, 'B': 1, 'C': 3, 'D': 4, 'E': 4}
  has skipped rank? True

{'A': 1}
  has skipped rank? False

{'A': 2}
  has skipped rank? True

{'A': 1, 'C': 2}
  has skipped rank? False

{'A': 1, 'B': 4}
  has skipped rank? True

{'A': 1, 'B': 1}
  has skipped rank? False

{'A': 2, 'B': 2}
  has skipped rank? True

{'A': 1, 'B': 1, 'C': 1, 'D': 1, 'E': 1}
  has skipped rank? False

{'A': 2, 'B': 2, 'C': 2, 'D': 2, 'E': 2}
  has skipped rank? True



In [4]:
def has_skipped_rank_above(ballot, cand):
    """Returns True if cand is ranked and there is a skipped rank above the rank of cand in ballot."""
    cand_rank = ballot.get(cand, None)
    if cand_rank is None:
        return False
    for rank in range(1, cand_rank):
        if rank not in ballot.values():
            return True
    return False

for b in ballots: 
    print(b)
    print("  has skipped rank above A?", has_skipped_rank_above(b, A))
    print("  has skipped rank above B?", has_skipped_rank_above(b, B))
    print("  has skipped rank above C?", has_skipped_rank_above(b, C))
    print("  has skipped rank above D?", has_skipped_rank_above(b, D))
    print("  has skipped rank above E?", has_skipped_rank_above(b, E))
    print()

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
  has skipped rank above A? False
  has skipped rank above B? False
  has skipped rank above C? False
  has skipped rank above D? False
  has skipped rank above E? False

{'A': 1, 'B': 1, 'C': 3, 'D': 4, 'E': 4}
  has skipped rank above A? False
  has skipped rank above B? False
  has skipped rank above C? True
  has skipped rank above D? True
  has skipped rank above E? True

{'A': 1}
  has skipped rank above A? False
  has skipped rank above B? False
  has skipped rank above C? False
  has skipped rank above D? False
  has skipped rank above E? False

{'A': 2}
  has skipped rank above A? True
  has skipped rank above B? False
  has skipped rank above C? False
  has skipped rank above D? False
  has skipped rank above E? False

{'A': 1, 'C': 2}
  has skipped rank above A? False
  has skipped rank above B? False
  has skipped rank above C? False
  has skipped rank above D? False
  has skipped rank above E? False

{'A': 1, 'B': 4}
  has skipped r

In [5]:
def has_skipped_rank(ballot):
    """Returns True if there is any skipped rank in the ballot."""
    ranks = sorted(set(ballot.values()))
    return ranks != list(range(1, len(ranks)+1))

def has_skipped_rank_above(ballot, cand):
    """Returns True if cand is ranked and there is a skipped rank above the rank of cand in ballot."""
    cand_rank = ballot.get(cand, None)
    if cand_rank is None:
        return False
    for rank in range(1, cand_rank):
        if rank not in ballot.values():
            return True
    return False

def infer_pairwise_comparison_from_ballot_alaska_rules(ballot, cand1, cand2): 
    """Infers the pairwise comparison between cand1 and cand2 from ballot according tot he following rule: 

    - If both candidates are ranked, then the one with the lower rank is the pairwise choice; otherwise, they are both the pairwise choice (tie).
    - If only one candidate is ranked, then that candidate is the pairwise choice if there is not a skipped rank above that candidate; otherwise, the pairwise comparison is undetermined (None).
    - If neither candidate is ranked, then the pairwise comparison is undetermined (None).
    
    """
    rank_cand1 = ballot.get(cand1, None)
    rank_cand2 = ballot.get(cand2, None)
    menu = {cand1, cand2}
    if rank_cand1 is not None and rank_cand2 is not None:
        if rank_cand1 < rank_cand2:
            return (menu, {cand1}) # cand1 is chosen
        elif rank_cand2 < rank_cand1:
            return (menu, {cand2}) # cand2 is chosen
        else:
            return (menu, menu)  # tie
    # exactly one of the candidates is ranked
    elif (rank_cand1 is None or rank_cand2 is None) and not (rank_cand1 is None and rank_cand2 is None): 
        ranked_cand = cand1 if rank_cand1 is not None else cand2
        if not has_skipped_rank_above(ballot, ranked_cand):
            return (menu, {ranked_cand}) # ranked_cand is chosen
        else:
            return None # undetermined
    else: # neither candidate is ranked
        if not has_skipped_rank(ballot):
            return (menu, menu) # tie
        else:
            return None # undetermined


In [9]:
for b in ballots:
    print("Original ballot:", b)
    for c1 in cands:
        for c2 in cands:
            if c1 < c2:
                print(f"  {c1} vs. {c2}: ", infer_pairwise_comparison_from_ballot_alaska_rules(b, c1, c2))
    print()

Original ballot: {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
  A vs. B:  ({'B', 'A'}, {'A'})
  A vs. C:  ({'C', 'A'}, {'A'})
  A vs. D:  ({'D', 'A'}, {'A'})
  A vs. E:  ({'E', 'A'}, {'A'})
  B vs. C:  ({'B', 'C'}, {'B'})
  B vs. D:  ({'D', 'B'}, {'B'})
  B vs. E:  ({'E', 'B'}, {'B'})
  C vs. D:  ({'D', 'C'}, {'C'})
  C vs. E:  ({'E', 'C'}, {'C'})
  D vs. E:  ({'E', 'D'}, {'D'})

Original ballot: {'A': 1, 'B': 1, 'C': 3, 'D': 4, 'E': 4}
  A vs. B:  ({'B', 'A'}, {'B', 'A'})
  A vs. C:  ({'C', 'A'}, {'A'})
  A vs. D:  ({'D', 'A'}, {'A'})
  A vs. E:  ({'E', 'A'}, {'A'})
  B vs. C:  ({'B', 'C'}, {'B'})
  B vs. D:  ({'D', 'B'}, {'B'})
  B vs. E:  ({'E', 'B'}, {'B'})
  C vs. D:  ({'D', 'C'}, {'C'})
  C vs. E:  ({'E', 'C'}, {'C'})
  D vs. E:  ({'E', 'D'}, {'E', 'D'})

Original ballot: {'A': 1}
  A vs. B:  ({'B', 'A'}, {'A'})
  A vs. C:  ({'C', 'A'}, {'A'})
  A vs. D:  ({'D', 'A'}, {'A'})
  A vs. E:  ({'E', 'A'}, {'A'})
  B vs. C:  ({'B', 'C'}, {'B', 'C'})
  B vs. D:  ({'D', 'B'}, {'D', 'B'})
  B 

In [ ]:
for b in ballots:
    print(b)
    pb = ballot_to_pairwise(b, cands)
    print("  ", pb)
    print("    is transitive?", pb.is_transitive(cands))
    print("    is coherent", pb.is_coherent())
    print("    is empty", pb.is_empty())
    if pb.is_coherent() and pb.is_transitive(cands):
        print("    ranking: ", pb.to_ranking())
    for c in cands:
        print(f"  strict preference of {A} over {c}? {pb.strict_pref(A, c)} ")
    print()

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}


NameError: name 'ballot_to_pairwise' is not defined

In [8]:
pprof = PairwiseProfile([ballot_to_pairwise(b, cands) for b in ballots], candidates=cands)

pprof.display()

MWSL(pprof)

for c in cands:
    for d in cands:
        if c != d:
            print(f"Majority margin of {c} over {d}: {pprof.margin(c, d)}")
            print(f" Support for {c} over {d}: {pprof.support(c, d)}")
            print(f" Support for {d} over {c}: {pprof.support(d, c)}\n")
pprof.margin_graph().display()
print(pprof.num_voters)



NameError: name 'ballot_to_pairwise' is not defined

In [105]:
print(sum([c for b,c in zip(*pprof.comparisons_counts) if b.is_empty()]))

print(sum([c for b,c in zip(*pprof.comparisons_counts) if b.is_transitive(pprof.candidates) and b.is_coherent() and not b.is_empty() and b.to_ranking().is_bullet_vote() ]))


print(sum([c for b,c in zip(*pprof.comparisons_counts) if b.is_transitive(pprof.candidates) and b.is_coherent() and not b.is_empty() and b.to_ranking().is_linear(len(pprof.candidates)) ]))

print(sum([c for b,c in zip(*pprof.comparisons_counts) if b.is_transitive(pprof.candidates) and b.is_coherent() and not b.is_empty() and b.to_ranking().has_tie()]))

print(sum([c for b,c in zip(*pprof.comparisons_counts) if b.has_tie()]))



0
0
0
3
3


In [106]:
r = Ranking({
    A:1,
    B:3,
    C:3,
    D:3, 
})
print(r.is_bullet_vote())

len(r.first()) == 1 and len(r.ranks) ==2

True


True

In [107]:
r = Ranking({
    A:2,
    B:3,
})
print(r.is_truncated_linear(4))



True


In [108]:
def _is_truncated_linear(ranks, num_cands):

    if not ranks:
        return False
    
    # Must have fewer distinct ranks than num_cands (truncated condition)
    unique_ranks = set(ranks)
    if len(unique_ranks) >= num_cands:
        return False
    
    if len(unique_ranks) == len(ranks):
        return True  # A bullet vote is a truncate linear order
        
    # Check that ties only occur at the highest rank
    max_rank = max(ranks)
    rank_counts = {}
    for rank in ranks:
        rank_counts[rank] = rank_counts.get(rank, 0) + 1
    
    # All ranks except the max should appear exactly once (no ties)
    for rank, count in rank_counts.items():
        if rank != max_rank and count > 1:
            return False
    
    return True


In [109]:
rs = [3, 3, 1,7,  6]
print(_is_truncated_linear(rs, 5))  

False


In [110]:
r1 = Ranking({
    A:1,
    B:2,
    C:3,
    D:4,
    E:4
})
num_cands = 5
print(r1.is_truncated_linear(num_cands))
print(_is_truncated_linear(list(r1.rmap.values()), num_cands)  )

False
True


In [111]:
def trucate_ballot(ballot): 
    """Truncates the ballot at the first occurrence of two skipped ranks in a row."""
    if not ballot:
        return ballot
    ranks = sorted(set(ballot.values()))
    truncated_ranks = []
    prev_rank = 0
    skip_count = 0
    for ridx, rank in enumerate(ranks):
        next_rank = ranks[ridx + 1] if ridx + 1 < len(ranks) else None
        truncated_ranks.append(rank)
        if next_rank is not None and next_rank - rank - 1 >= 2:
            break
        
        
    
    truncated_ballot = {cand: rank for cand, rank in ballot.items() if rank in truncated_ranks}
    return truncated_ballot

b1 = {A:1, B:2, C:4, D:5}
b2 = {A:1, B:3, C:5}
b3 = {A:2, B:5}
b4 = {A:1, B:3, C:1, D:5, E:6}
b5 = {A:1, B:3, C:1, D:6, E:6}
b6 = {A:1, B:1, C:4, D:2, E:7}

ballots = [b1, b2, b3, b4, b5, b6]
for b in ballots:
    print("Original ballot:", b)
    print("Truncated ballot:", trucate_ballot(b))
    print()


Original ballot: {'A': 1, 'B': 2, 'C': 4, 'D': 5}
Truncated ballot: {'A': 1, 'B': 2, 'C': 4, 'D': 5}

Original ballot: {'A': 1, 'B': 3, 'C': 5}
Truncated ballot: {'A': 1, 'B': 3, 'C': 5}

Original ballot: {'A': 2, 'B': 5}
Truncated ballot: {'A': 2}

Original ballot: {'A': 1, 'B': 3, 'C': 1, 'D': 5, 'E': 6}
Truncated ballot: {'A': 1, 'B': 3, 'C': 1, 'D': 5, 'E': 6}

Original ballot: {'A': 1, 'B': 3, 'C': 1, 'D': 6, 'E': 6}
Truncated ballot: {'A': 1, 'B': 3, 'C': 1}

Original ballot: {'A': 1, 'B': 1, 'C': 4, 'D': 2, 'E': 7}
Truncated ballot: {'A': 1, 'B': 1, 'C': 4, 'D': 2}



In [112]:
def infer_pairwise_comparison_from_ballot_truncation_rules(ballot, cand1, cand2): 
    """Infers the pairwise comparison between cand1 and cand2 from ballot according tot he following rule: 

    - If both candidates are ranked, then the one with the lower rank is the pairwise choice; otherwise, they are both the pairwise choice (tie).
    - If there are two skips in a row, then the ballot is truncated at that rank.
    - If only one candidate is ranked, then that candidate is the pairwise choice.
    - If both candidates are unranked, then they are both the pairwise choice (tie).

    """
    truncated_ballot = trucate_ballot(ballot)
    rank_cand1 = truncated_ballot.get(cand1, None)
    rank_cand2 = truncated_ballot.get(cand2, None)
    menu = {cand1, cand2}
    if rank_cand1 is not None and rank_cand2 is not None:
        if rank_cand1 < rank_cand2:
            return (menu, {cand1}) # cand1 is chosen
        elif rank_cand2 < rank_cand1:
            return (menu, {cand2}) # cand2 is chosen
        else:
            return (menu, menu)  # tie
    # exactly one of the candidates is ranked
    elif (rank_cand1 is None or rank_cand2 is None) and not (rank_cand1 is None and rank_cand2 is None): 
        ranked_cand = cand1 if rank_cand1 is not None else cand2
        unranked_cand = cand2 if rank_cand1 is not None else cand1
        return (menu, {ranked_cand}) # ranked_cand is chosen
    else: # neither candidate is ranked
        return (menu, menu) # tie

cands = [A, B, C, D, E]
ballots = [b1, b2, b3, b4, b5, b6]
for b in ballots: 
    print(b)
    for c1 in cands:
        for c2 in cands:
            if c1 < c2:
                print(f"  {c1} vs {c2}: {infer_pairwise_comparison_from_ballot_truncation_rules(b, c1, c2)}")
    print()

{'A': 1, 'B': 2, 'C': 4, 'D': 5}
  A vs B: ({'B', 'A'}, {'A'})
  A vs C: ({'C', 'A'}, {'A'})
  A vs D: ({'D', 'A'}, {'A'})
  A vs E: ({'E', 'A'}, {'A'})
  B vs C: ({'B', 'C'}, {'B'})
  B vs D: ({'D', 'B'}, {'B'})
  B vs E: ({'E', 'B'}, {'B'})
  C vs D: ({'D', 'C'}, {'C'})
  C vs E: ({'E', 'C'}, {'C'})
  D vs E: ({'D', 'E'}, {'D'})

{'A': 1, 'B': 3, 'C': 5}
  A vs B: ({'B', 'A'}, {'A'})
  A vs C: ({'C', 'A'}, {'A'})
  A vs D: ({'D', 'A'}, {'A'})
  A vs E: ({'E', 'A'}, {'A'})
  B vs C: ({'B', 'C'}, {'B'})
  B vs D: ({'D', 'B'}, {'B'})
  B vs E: ({'E', 'B'}, {'B'})
  C vs D: ({'D', 'C'}, {'C'})
  C vs E: ({'E', 'C'}, {'C'})
  D vs E: ({'D', 'E'}, {'D', 'E'})

{'A': 2, 'B': 5}
  A vs B: ({'B', 'A'}, {'A'})
  A vs C: ({'C', 'A'}, {'A'})
  A vs D: ({'D', 'A'}, {'A'})
  A vs E: ({'E', 'A'}, {'A'})
  B vs C: ({'B', 'C'}, {'B', 'C'})
  B vs D: ({'D', 'B'}, {'D', 'B'})
  B vs E: ({'E', 'B'}, {'E', 'B'})
  C vs D: ({'D', 'C'}, {'D', 'C'})
  C vs E: ({'E', 'C'}, {'E', 'C'})
  D vs E: ({'D', 'E'}, 

In [113]:

def ballot_to_pairwise(ballot, candidates, ballot_to_comparison_fnc):
    comparisons = []
    for c1 in candidates:
        for c2 in candidates:
            if c1 != c2:
                comparison = ballot_to_comparison_fnc(ballot, c1, c2)
                if comparison is not None:
                    if all([comparison[0] != menu for menu, _ in comparisons]):
                        comparisons.append(comparison)

    return PairwiseBallot(comparisons, candidates=candidates)

def to_pairwise_profile(ballots, candidates, ballot_to_comparison_fnc):
    return PairwiseProfile([ballot_to_pairwise(b, candidates, ballot_to_comparison_fnc) for b in ballots], candidates=candidates)

In [114]:
for b in ballots:
    print(b)
    pb = ballot_to_pairwise(b, cands, infer_pairwise_comparison_from_ballot_truncation_rules)
    print("  ", pb)
    print("    is transitive?", pb.is_transitive(cands))
    print("    is coherent", pb.is_coherent())
    print("    is empty", pb.is_empty())
    if pb.is_coherent() and pb.is_transitive(cands):
        print("    ranking: ", pb.to_ranking())
    for c in cands:
        print(f"  strict preference of {A} over {c}? {pb.strict_pref(A, c)} ")
    print()

    pprof = to_pairwise_profile(ballots, cands, infer_pairwise_comparison_from_ballot_truncation_rules)

    pprof.display()

{'A': 1, 'B': 2, 'C': 4, 'D': 5}
   {A, B} -> {A}, {A, C} -> {A}, {A, D} -> {A}, {A, E} -> {A}, {B, C} -> {B}, {B, D} -> {B}, {B, E} -> {B}, {C, D} -> {C}, {C, E} -> {C}, {D, E} -> {D}
    is transitive? True
    is coherent True
    is empty False
    ranking:  A B C D E 
  strict preference of A over A? False 
  strict preference of A over B? True 
  strict preference of A over C? True 
  strict preference of A over D? True 
  strict preference of A over E? True 

1: {A, B} -> {A}, {A, C} -> {A}, {A, D} -> {A}, {A, E} -> {A}, {B, C} -> {B}, {B, D} -> {B}, {B, E} -> {B}, {C, D} -> {C}, {C, E} -> {C}, {D, E} -> {D}
1: {A, B} -> {A}, {A, C} -> {A}, {A, D} -> {A}, {A, E} -> {A}, {B, C} -> {B}, {B, D} -> {B}, {B, E} -> {B}, {C, D} -> {C}, {C, E} -> {C}, {D, E} -> {D, E}
1: {A, B} -> {A}, {A, C} -> {A}, {A, D} -> {A}, {A, E} -> {A}, {B, C} -> {B, C}, {B, D} -> {B, D}, {B, E} -> {B, E}, {C, D} -> {C, D}, {C, E} -> {C, E}, {D, E} -> {D, E}
1: {A, B} -> {A}, {A, C} -> {A, C}, {A, D} -> {A}, {

In [115]:
print(cands)
for b in ballots:
    print(b)
    pb = ballot_to_pairwise(b, cands, infer_pairwise_comparison_from_ballot_alaska_rules)
    print("  ", pb)
    print("    is transitive?", pb.is_transitive(cands))
    print("    is coherent", pb.is_coherent())
    print("    is empty", pb.is_empty())
    if pb.is_coherent() and pb.is_transitive(cands):
        print("    ranking: ", pb.to_ranking())
    for c in cands:
        print(f"  strict preference of {A} over {c}? {pb.strict_pref(A, c)} ")
    for c in cands:
        print(f"  strict preference of {C} over {c}? {pb.strict_pref(A, c)} ")
    print()
pprof = to_pairwise_profile(ballots, cands, infer_pairwise_comparison_from_ballot_alaska_rules)
pprof.display()

['A', 'B', 'C', 'D', 'E']
{'A': 1, 'B': 2, 'C': 4, 'D': 5}
   {A, B} -> {A}, {A, C} -> {A}, {A, D} -> {A}, {A, E} -> {A}, {B, C} -> {B}, {B, D} -> {B}, {B, E} -> {B}, {C, D} -> {C}
    is transitive? True
    is coherent False
    is empty False
  strict preference of A over A? False 
  strict preference of A over B? True 
  strict preference of A over C? True 
  strict preference of A over D? True 
  strict preference of A over E? True 
  strict preference of C over A? False 
  strict preference of C over B? True 
  strict preference of C over C? True 
  strict preference of C over D? True 
  strict preference of C over E? True 

{'A': 1, 'B': 3, 'C': 5}
   {A, B} -> {A}, {A, C} -> {A}, {A, D} -> {A}, {A, E} -> {A}, {B, C} -> {B}
    is transitive? True
    is coherent False
    is empty False
  strict preference of A over A? False 
  strict preference of A over B? True 
  strict preference of A over C? True 
  strict preference of A over D? True 
  strict preference of A over E? True